<a href="https://colab.research.google.com/github/Iqra-01/Google_AI_studio_Genai/blob/main/templates/structured%20trip%20recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
!pip install -U -q "google-generativeai>=0.8.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 20.6 MB/s eta 0:00:00


In [2]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IkhlcmUgaXMgYW4gYXJ0aWNsZSBhYm91dCBleGNpdGluZyB0aGluZ3MgdG8gZG8gaW4gTmV3IFlvcmsgQ2l0eS4gQ29udmVydCB0aGlzIGludG8gYSBzdHJ1Y3R1cmVkIGZvcm1hdCB3aXRoIDMgY29sdW1ucyAtIGFjdGl2aXR5LCBiZXN0IHNlYXNvbiB0byBwYXJ0YWtlIGluIHRoaXMgYWN0aXZpdHksIGFuZCBhbW91bnQgb2YgdGltZSBuZWVkZWQuXFxuXFxuTmV3IFlvcmsgQ2l0eSB2aWJyYXRlcyB3aXRoIGFuIGlycmVzaXN0aWJsZSBlbmVyZ3kgdGhhdCBuZXZlciBzbGVlcHMsIG9mZmVyaW5nIGFuIGFidW5kYW5jZSBvZiBleHBlcmllbmNlcyBmb3IgZXZlcnkgdHJhdmVsZXIhIEluIHRoZSBwbGVhc2FudCBlbWJyYWNlIG9mIHNwcmluZyBvciB0aGUgY29sb3JmdWwgdGFwZXN0cnkgb2YgZmFsbCwgZGVkaWNhdGUgYSBoYWxmIG9yIGV2ZW4gYSBmdWxsIGRheSB0byBleHBsb3JpbmcgdGhlIHNwcmF3bGluZyBleHBhbnNlIG9mIENlbnRyYWwgUGFyay4gVGFrZSBhIGxlaXN1cmVseSBiaWtlIHJpZGUsIGVuam95IGEgcGljbmljIGx1bmNoLCBvciByZW50IGEgYm9hdCBvbiB0aGUgbGFrZSDigJMgdGhlIHBvc3NpYmlsaXRpZXMgZm9yIGVuam95bWVudCBhcmUgZW5kbGVzcyEgUmVnYXJkbGVzcyBvZiB0aGUgc2Vhc29uLCBhcnQgZW50aHVzaWFzdHMgY2FuIGltbWVyc2UgdGhlbXNlbHZlcyBpbiB0aGUgdmFzdCBjb2xsZWN0aW9uIG9mIHRoZSBNZXRyb3BvbGl0YW4gTXVzZXVtIG9mIEFydCwgZGVkaWNhdGluZyBhbiBhZnRlcm5vb24gdG8gd2FuZGVyIHRocm91Z2ggY2VudHVyaWVzIG9mIGh1bWFuIGNyZWF0aXZpdHkuXFxuXFxuRm9yIGEgZG9zZSBvZiBjYXB0aXZhdGluZyBlbnRlcnRhaW5tZW50LCBjYXRjaCBhIHdvcmxkLXJlbm93bmVkIEJyb2Fkd2F5IHNob3csIGEgdGhyZWUtaG91ciBqb3VybmV5IGludG8gdGhlIGhlYXJ0IG9mIHRoZWF0ZXIgYW5kIG11c2ljLiBTcHJpbmcgb3IgZmFsbCBvZmZlcnMgdGhlIGlkZWFsIHdlYXRoZXIgZm9yIGEgZmVycnkgcmlkZSB0byB0aGUgU3RhdHVlIG9mIExpYmVydHkgYW5kIEVsbGlzIElzbGFuZCwgYWxsb3dpbmcgZm9yIGEgaGFsZiBvciBmdWxsIGRheSB0byBkZWx2ZSBpbnRvIHRoZSBzdG9yaWVzIG9mIGltbWlncmF0aW9uIGFuZCBmcmVlZG9tIHRoYXQgc2hhcGVkIEFtZXJpY2EuIFdoZW4gdGhlIHdlYXRoZXIgYmVja29ucyB5b3Ugb3V0ZG9vcnMgaW4gdGhlIHN1bW1lciBtb250aHMsIGRlZGljYXRlIGFuIGhvdXIgdG8gYSBzdHJvbGwgYWxvbmcgdGhlIEhpZ2ggTGluZSwgYSB1bmlxdWUgZWxldmF0ZWQgcGFyayBvZmZlcmluZyBicmVhdGh0YWtpbmcgY2l0eSB2aWV3cyBhbmQgdXJiYW4gZ2FyZGVucy5cXG5cXG5BcyBkdXNrIHNldHRsZXMsIGhlYWQgdG8gdGhlIGRhenpsaW5nIHNwZWN0YWNsZSBvZiBUaW1lcyBTcXVhcmUsIHNwZW5kaW5nIHRoaXJ0eSBtaW51dGVzIHRvIGFuIGhvdXIgc29ha2luZyBpbiB0aGUgdmlicmFudCBsaWdodHMgYW5kIGJ1c3RsaW5nIGVuZXJneS4gRm9yIGEgbW9tZW50IG9mIHJlZmxlY3Rpb24sIHZpc2l0IHRoZSBwb2lnbmFudCA5LzExIE1lbW9yaWFsICYgTXVzZXVtLCBkZWRpY2F0aW5nIHR3byB0byB0aHJlZSBob3VycyB0byBob25vciB0aGUgbGl2ZXMgbG9zdCBhbmQgdGhlIHVud2F2ZXJpbmcgc3Bpcml0IG9mIHRoZSBjaXR5LiBGcm9tIHRoZSB0cmVuZHkgc3RyZWV0cyBvZiBTb2hvIHRvIHRoZSBoaXN0b3JpYyBjaGFybSBvZiBHcmVlbndpY2ggVmlsbGFnZSwgZXhwbG9yaW5nIHRoZSBkaXZlcnNlIG5laWdoYm9yaG9vZHMgb2YgTmV3IFlvcmsgQ2l0eSBpcyBhbiBhZHZlbnR1cmUgaW4gaXRzZWxmLCB3aXRoIHRoZSB0aW1lIGNvbW1pdG1lbnQgdmFyeWluZyBiYXNlZCBvbiB5b3VyIGNob3NlbiBhcmVhIGFuZCBpbnRlcmVzdHMuIEFuZCBmb3IgdGhvc2Ugc2Vla2luZyBhIHRhc3RlIG9mIHRoZSBjaXR5J3MgdmlicmFudCBjdWx0dXJlLCBhIHR3by1ob3VyIGhvdXIgZm9vZCB0b3VyIHdpbGwgdGFudGFsaXplIHlvdXIgdGFzdGUgYnVkcyB3aXRoIGRpdmVyc2UgZmxhdm9ycyBhbmQgY3VsaW5hcnkgZGVsaWdodHMuXFxuXFxuTm8gbWF0dGVyIHlvdXIgaW50ZXJlc3RzIG9yIHRoZSB0aW1lIG9mIHllYXIsIE5ldyBZb3JrIENpdHkgcHJvbWlzZXMgYW4gdW5mb3JnZXR0YWJsZSBleHBlcmllbmNlLCBicmltbWluZyB3aXRoIGljb25pYyBzaWdodHMsIGNhcHRpdmF0aW5nIHBlcmZvcm1hbmNlcywgYW5kIGVuZGxlc3Mgb3Bwb3J0dW5pdGllcyBmb3IgZXhwbG9yYXRpb24hIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoifCBBY3Rpdml0eSAgICAgICAgICAgICAgICAgICAgIHwgQmVzdCBTZWFzb24gdG8gUGFydGFrZSB8IFRpbWUgTmVlZGVkICAgICAgICAgICB8XFxufC0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLXwtLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tfC0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tfFxcbnwgRXhwbG9yZSBDZW50cmFsIFBhcmsgICAgICAgICB8IFNwcmluZywgRmFsbCAgICAgICAgICAgIHwgSGFsZi1kYXkgdG8gRnVsbC1kYXkgICB8XFxufCBWaXNpdCBNZXRyb3BvbGl0YW4gTXVzZXVtIG9mIEFydCB8IEFueSBTZWFzb24gICAgICAgICAgICAgIHwgQWZ0ZXJub29uICAgICAgICAgICAgIHxcXG58IEF0dGVuZCBhIEJyb2Fkd2F5IFNob3cgICAgICAgfCBBbnkgU2Vhc29uICAgICAgICAgICAgICB8IDMgaG91cnMgICAgICAgICAgICAgICAgfFxcbnwgRmVycnkgdG8gU3RhdHVlIG9mIExpYmVydHkgJiBFbGxpcyBJc2xhbmQgfCBTcHJpbmcsIEZhbGwgICAgICAgICAgICB8IEhhbGYtZGF5IHRvIEZ1bGwtZGF5ICAgfFxcbnwgU3Ryb2xsIGFsb25nIHRoZSBIaWdoIExpbmUgICB8IFN1bW1lciAgICAgICAgICAgICAgICAgIHwgMSBob3VyICAgICAgICAgICAgICAgICB8XFxufCBWaXNpdCBUaW1lcyBTcXVhcmUgICAgICAgICAgIHwgQW55IFNlYXNvbiAgICAgICAgICAgICAgfCAzMCBtaW51dGVzIC0gMSBob3VyICAgIHxcXG58IFZpc2l0IDkvMTEgTWVtb3JpYWwgJiBNdXNldW0gfCBBbnkgU2Vhc29uICAgICAgICAgICAgICB8IDItMyBob3VycyAgICAgICAgICAgICAgfFxcbnwgRXhwbG9yZSBOWUMgTmVpZ2hib3Job29kcyAoZS5nLiwgU29obywgR3JlZW53aWNoIFZpbGxhZ2UpIHwgQW55IFNlYXNvbiAgICAgICAgICAgICAgfCBWYXJpZXMgICAgICAgICAgICAgICAgICB8XFxufCBUYWtlIGEgRm9vZCBUb3VyICAgICAgICAgICAgIHwgQW55IFNlYXNvbiAgICAgICAgICAgICAgfCAyIGhvdXJzICAgICAgICAgICAgICAgIHxcXG5cXG4ifV19XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo2NCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "Here is an article about exciting things to do in New York City. Convert this into a structured format with 3 columns - activity, best season to partake in this activity, and amount of time needed.\\n\\nNew York City vibrates with an irresistible energy that never sleeps, offering an abundance of experiences for every traveler! In the pleasant embrace of spring or the colorful tapestry of fall, dedicate a half or even a full day to exploring the sprawling expanse of Central Park. Take a leisurely bike ride, enjoy a picnic lunch, or rent a boat on the lake \u2013 the possibilities for enjoyment are endless! Regardless of the season, art enthusiasts can immerse themselves in the vast collection of the Metropolitan Museum of Art, dedicating an afternoon to wander through centuries of human creativity.\\n\\nFor a dose of captivating entertainment, catch a world-renowned Broadway show, a three-hour jou

## Call `generate_content`

In [4]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

if response.candidates:
    display(Markdown(response.text))
else:
    print("The model did not return any candidates. Please check your prompt and configuration.")
    # Optionally, you can print more details about the response:
    print(f"Response details: {response}")

The model did not return any candidates. Please check your prompt and configuration.
Response details: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "usage_metadata": {
        "prompt_token_count": 649,
        "total_token_count": 649
      }
    }),
)


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [5]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

Here is an article about exciting things to do in New York City. Convert this into a structured format with 3 columns - activity, best season to partake in this activity, and amount of time needed.\n\nNew York City vibrates with an irresistible energy that never sleeps, offering an abundance of experiences for every traveler! In the pleasant embrace of spring or the colorful tapestry of fall, dedicate a half or even a full day to exploring the sprawling expanse of Central Park. Take a leisurely bike ride, enjoy a picnic lunch, or rent a boat on the lake – the possibilities for enjoyment are endless! Regardless of the season, art enthusiasts can immerse themselves in the vast collection of the Metropolitan Museum of Art, dedicating an afternoon to wander through centuries of human creativity.\n\nFor a dose of captivating entertainment, catch a world-renowned Broadway show, a three-hour journey into the heart of theater and music. Spring or fall offers the ideal weather for 